In [1]:
import numpy as np
import itertools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
import gc
gc.collect()
from sklearn.linear_model import SGDRegressor, PassiveAggressiveRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error
from copy import copy
from sklearn.decomposition import IncrementalPCA
from multiprocessing import Process
from joblib import load, dump, Memory
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import glob2
import numba
import scipy.stats as sp
import sys
from skgarden.mondrian import MondrianTreeRegressor, MondrianForestRegressor
from skgarden.forest import ExtraTreesRegressor as ETR
import math
import os
from os import system, name
from pathlib import Path
from sklearn.feature_selection import SelectFwe, f_regression, SelectPercentile
from sklearn.preprocessing import LabelEncoder
import logging
import warnings
from scipy import stats
from skgarden.quantile import DecisionTreeQuantileRegressor, ExtraTreeQuantileRegressor,ExtraTreesQuantileRegressor,RandomForestQuantileRegressor
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor,BaggingRegressor
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
import math
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.utils import check_random_state
from sklearn.multioutput import MultiOutputRegressor
import sklearn
from sklearn.decomposition import PCA
import dask as dd
import statistics
import numexpr as ne

In [ ]:
"""""""""""""""""""""""""""""METRICS"""""""""""""""
#@numba.jit(parallel=True,nopython=True, cache=True, fastmath=True, nogil=True)
def mae_new(y_true, y_pred, sample_weight=None):
    return np.mean(ne.evaluate("abs((y_true)-(y_pred))"))
def mae(y_true, y_pred, sample_weight=None):
    return mae_new(y_true, y_pred, sample_weight=None)

#@numba.jit(parallel=True,nopython=True, cache=True, fastmath=True, nogil=True)
def mse_new(y_true, y_pred, sample_weight=None):
    return np.mean(ne.evaluate("((y_true)-(y_pred))**2"))
def mse(y_true, y_pred, sample_weight=None):
    return mse_new(y_true, y_pred, sample_weight=None)
def mdse_new(y_true, y_pred, sample_weight=None):
    return np.median(ne.evaluate("((y_true)-(y_pred))**2"))
def mdse(y_true, y_pred, sample_weight=None):
    return mdse_new(y_true, y_pred, sample_weight=None)
def msle_new(y_true, y_pred, sample_weight=None):
    return np.mean(ne.evaluate("(log1p(y_true)-log1p(y_pred))**2"))
def msle(y_true, y_pred, sample_weight=None):
    return msle_new(y_true, y_pred, sample_weight=None)
def mdlse_new(y_true, y_pred, sample_weight=None):
    return np.median(ne.evaluate("(log1p(y_true)-log1p(y_pred))**2"))
def mdlse(y_true, y_pred, sample_weight=None):
    return mdlse_new(y_true, y_pred, sample_weight=None)
#@numba.jit(parallel=True,nopython=True, cache=True, fastmath=True, nogil=True)
def rmse_new(y_true, y_pred, sample_weight=None):
    new=np.mean(ne.evaluate("((y_true)-(y_pred))**2"))
    return ne.evaluate("sqrt(new)")
def rmse(y_true, y_pred, sample_weight=None):
    return rmse_new(y_true, y_pred, sample_weight=None)

# vectorized error calc
#@numba.jit(parallel=True,nopython=True, cache=True, fastmath=True, nogil=True)
def rmdse_new(y_true, y_pred, sample_weight=None):
    new=np.median(ne.evaluate("((y_true)-(y_pred))**2"))
    return ne.evaluate("sqrt(new)")
def rmdse(y_true, y_pred, sample_weight=None):
    return rmdse_new(y_true, y_pred, sample_weight=None)

# vectorized error calc
#@numba.jit(parallel=True,nopython=True, cache=True, fastmath=True, nogil=True)
def mde_new(y_true, y_pred, sample_weight=None):
    return np.median(ne.evaluate("abs((y_true)-(y_pred))"))
def mde(y_true, y_pred, sample_weight=None):
    return mde_new(y_true, y_pred, sample_weight=None)

# vectorized error calc
#@numba.jit(parallel=True,nopython=True, cache=True, fastmath=True, nogil=True)
def rmsle_new(y_true, y_pred, sample_weight=None):
    new=np.mean(ne.evaluate("(log1p(y_true)-log1p(y_pred))**2"))
    return ne.evaluate("sqrt(new)")
def rmsle(y_true, y_pred, sample_weight=None):
    return rmsle_new(y_true, y_pred, sample_weight=None)

# vectorized error calc
#@numba.jit(parallel=True,nopython=True, cache=True, fastmath=True, nogil=True)
def rmdsle_new(y_true, y_pred, sample_weight=None):
    new=np.median(ne.evaluate("(log1p(y_true)-log1p(y_pred))**2"))
    return ne.evaluate("sqrt(new)")
def rmdsle(y_true, y_pred, sample_weight=None):
    return rmdsle_new(y_true, y_pred, sample_weight=None)

In [ ]:
#@numba.jit(parallel=True,nopython=True, cache=True, fastmath=True, nogil=True)
def mape_new(y_true, y_pred, sample_weight=None):
    eps = 1E-15
    ape = ne.evaluate("abs((y_true - y_pred) / (y_true + eps)) * 100")
    return np.mean(ape)
def mape(y_true, y_pred, sample_weight=None):
    return mape_new(y_true, y_pred, sample_weight=None)

def mdape_new(y_true, y_pred, sample_weight=None):
    eps = 1E-15
    ape = ne.evaluate("abs((y_true - y_pred) / (y_true + eps)) * 100")
    return np.median(ape)
def mdape(y_true, y_pred, sample_weight=None):
    return mdape_new(y_true, y_pred, sample_weight=None)
def smape_new(y_true, y_pred, sample_weight=None):
    eps = 1E-15
    sape = ne.evaluate("(abs(y_true - y_pred) / (0.5 * (abs(y_true) + abs(y_pred)) + eps)) * 100")
    return np.mean(sape) 
def smape(y_true, y_pred, sample_weight=None):
    return smape_new(y_true, y_pred, sample_weight=None)
def smdpe_new(y_true, y_pred, sample_weight=None):
    eps = 1E-15
    sape = ne.evaluate("(abs(y_true - y_pred) / (0.5 * (abs(y_true) + abs(y_pred)) + eps)) * 100")
    return np.median(sape) 
def smdpe(y_true, y_pred, sample_weight=None):
    return smdpe_new(y_true, y_pred, sample_weight=None)

In [4]:
from dask_ml.wrappers import Incremental
from dask_ml.wrappers import ParallelPostFit
from sklearn.pipeline import make_union
from sklearn.base import TransformerMixin
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from joblib import Parallel, delayed
from catboost import CatBoostRegressor


#implementation of a feedforward neural network made of gradient boosted trees and extremely randomized trees in parallel. 
#key items are kept each layer and new layer is trained again to focus on the patterns that were found originaly and to focus now on find a good correlation 
#between the models made and next layers/ i.e. if you knew the outcome in parallel universe and wanted to change the outcome in your own. if you knew the outcome from each universe
# you would reach a point of convergence where cothing can be done no matter what you do
class MultiverseRegressor(BaseEstimator,RegressorMixin):
    def __init__(self, folds=2, metric="rmse", Incremental=True,restacking=True,learning_rate=0.0001, retrain=False, seed=12345, verbose=0, iterations=20000, parallel=1):
        #This algortihm creates parallel stacks of different types of trees to and aggregates them into a robust incremental super-learner
        self.FOLDS=folds
        self.METRIC=metric
        self.RESTACKING=restacking
        self.incremental=Incremental
        self.RETRAIN=retrain
        self.SEED=seed
        self.VERBOSE=verbose  
        self.fitted=False
        self.forest1=MultiOutputRegressor(MondrianForestRegressor(n_estimators=10))
        self.forest2=MultiOutputRegressor(MondrianForestRegressor(n_estimators=10))
        self.grad = MultiOutputRegressor(MondrianForestRegressor(n_estimators=10))
        self.PCA=PCA(n_components=12, copy=True)
        self.PCA1=PCA(n_components=8, copy=True)
        self.stack0=[
        MultiOutputRegressor(MondrianForestRegressor(n_estimators=10)),
        MultiOutputRegressor(LGBMRegressor(boosting_type='gbdt',learning_rate=learning_rate,shuffle=True, tree_learner='data',stratified=True, obective='rmse',n_estimators=iterations, max_depth=parallel,n_jobs=-1),n_jobs=-1),
        MultiOutputRegressor(CatBoostRegressor(iterations=iterations,eta=learning_rate,loss_function='RMSE', depth=parallel,save_snapshot=True, snapshot_file='level1.cbsnapshot'), n_jobs=-1),       
        MultiOutputRegressor(ExtraTreesRegressor(n_estimators=150,warm_start=True,criterion="friedman_mse",n_jobs=-1),n_jobs=-1),
        MultiOutputRegressor(XGBRegressor(booster='gbtree',tree_learner='hist', n_estimators=iterations,n_jobs=-1),n_jobs=-1),
        ]
        self.stack1=[
        MultiOutputRegressor(MondrianForestRegressor(n_estimators=10)),
        MultiOutputRegressor(LGBMRegressor(boosting_type='gbdt',learning_rate=learning_rate,shuffle=True, tree_learner='data',stratified=True, obective='rmse',n_estimators=iterations, max_depth=parallel,n_jobs=-1),n_jobs=-1),
        MultiOutputRegressor(ExtraTreesRegressor(n_estimators=150,warm_start=True,criterion="friedman_mse",n_jobs=-1),n_jobs=-1),
        MultiOutputRegressor(XGBRegressor(booster='gbtree',tree_learner='hist', n_estimators=iterations,n_jobs=-1),n_jobs=-1),
        ]
        self.stack2=[
        MultiOutputRegressor(MondrianForestRegressor(n_estimators=10)),
        MultiOutputRegressor(LGBMRegressor(boosting_type='gbdt',learning_rate=learning_rate,shuffle=True, tree_learner='data',stratified=True, obective='rmse',n_estimators=iterations, max_depth=parallel,n_jobs=-1),n_jobs=-1),
        MultiOutputRegressor(ExtraTreesRegressor(n_estimators=150,warm_start=True,criterion="friedman_mse",n_jobs=-1),n_jobs=-1),
        MultiOutputRegressor(XGBRegressor(booster='gbtree',tree_learner='hist', n_estimators=iterations,n_jobs=-1),n_jobs=-1),
        ]
    #fitting and predicting and fitting in parallel, will work even better on multicore CPU's with multithreading, highly scalable, with 3 CPUs, cuts time to a third
    def fit(self,X,y):
        if self.fitted==False:
            Parallel(n_jobs=-1, prefer="threads", require='sharedmem')(delayed(estimator.fit)(X,y) for estimator in self.stack0)
            self.o=(Parallel(n_jobs=-1, prefer="threads", require='sharedmem')(delayed(estimator.predict)(X) for estimator in self.stack0))
            newx=self.PCA.fit_transform(X)
            self.o=np.hstack((self.o))
            #incremental layer learning, forests are better at categorical learning, so my categories are predictions
            self.forest1.partial_fit(np.hstack((self.o,newx)),y)
            f1=self.forest1.predict(np.hstack((self.o,newx)))
            self.a=np.hstack((self.o,X,f1))
            Parallel(n_jobs=-1, prefer="threads", require='sharedmem')(delayed(estimator.fit)(self.a,y) for estimator in self.stack1)
            self.b=(Parallel(n_jobs=-1, prefer="threads", require='sharedmem')(delayed(estimator.predict)(self.a) for estimator in self.stack1))
            self.b=np.hstack((self.b))
            newx2=self.PCA1.fit_transform(X)
            #incremental layer learning
            self.forest2.partial_fit(np.hstack((self.b,newx2)),y)
            f2=self.forest2.predict(np.hstack((self.b,newx2)))
            self.b=np.hstack((self.a,self.b,f2))
            Parallel(n_jobs=-1, prefer="threads", require='sharedmem')(delayed(estimator.fit)(self.b,y) for estimator in self.stack2)
            self.c=(Parallel(n_jobs=-1, prefer="threads", require='sharedmem')(delayed(estimator.predict)(self.b) for estimator in self.stack2))
            self.c=np.hstack((self.c))
            #newx=self.PCA2.fit_transform(X)
            self.c=np.hstack((newx,self.o,f1,self.b,self.c))
            self.fitted=True
            self.grad.fit((self.c),y)
        else:
            self.o=(Parallel(n_jobs=-1,prefer="threads", require='sharedmem')(delayed(estimator.predict)(X) for estimator in self.stack0))
            self.o=np.hstack(self.o)
            newx=self.PCA.transform(X)
            self.forest1.partial_fit(np.hstack((self.o,newx)),y)
            f1=self.forest1.predict(np.hstack((self.o,newx)))
            self.a=np.hstack((self.o,X,f1))
            self.b=(Parallel(n_jobs=-1, prefer="threads", require='sharedmem')(delayed(estimator.predict)(self.a) for estimator in self.stack1))
            self.b=np.hstack((self.b))
            newx2=self.PCA1.transform(X)
            self.forest2.partial_fit(np.hstack((self.b,newx2)),y)
            f2=self.forest2.predict(np.hstack((self.b,newx2)))
            self.b=np.hstack((self.a,self.b,f2))
            self.c=(Parallel(n_jobs=-1, prefer="threads", require='sharedmem')(delayed(estimator.predict)(self.b) for estimator in self.stack2))
            self.c=np.hstack((self.c))
            #newx=self.PCA2.transform(X)
            self.c=np.hstack((newx,self.o,f1,self.b,self.c))
            self.grad.partial_fit((self.c),y)
        return self
    def partial_fit(self,X,y):
        return self.fit(X,y)
    def predict(self,X):
        self.o=(Parallel(n_jobs=-1,prefer="threads", require='sharedmem')(delayed(estimator.predict)(X) for estimator in self.stack0))
        self.o=np.hstack(self.o)
        newx=self.PCA.transform(X)
        f1=self.forest1.predict(np.hstack((self.o,newx)))
        self.a=np.hstack((self.o,X,f1))
        self.b=(Parallel(n_jobs=-1, prefer="threads", require='sharedmem')(delayed(estimator.predict)(self.a) for estimator in self.stack1)) 
        self.b=np.hstack((self.b))
        newx2=self.PCA1.transform(X)
        f2=self.forest2.predict(np.hstack((self.b,newx2)))
        self.b=np.hstack((self.a,self.b,f2))
        self.c=(Parallel(n_jobs=-1, prefer="threads", require='sharedmem')(delayed(estimator.predict)(self.b) for estimator in self.stack2))
        self.c=np.hstack((self.c))
        #newx=self.PCA2.fit_transform(X)
        self.c=np.hstack((newx,self.o,f1,self.b,self.c))
        return self.grad.predict((self.c))

In [5]:
#clear previous data
warnings.filterwarnings("ignore")
from threading import Thread


def clear():
    # for windows
    if name == 'nt':
        _ = system('cls')
    else:
        _ = system('clear')


clear()
path = os.getcwd()
INPUT_DIRTR = (str(path) + '/DB/Train')
INPUT_DIRTX = (str(path) + '/DB/Test2')

# garbage collector
import gc
gc.enable()


In [6]:
"""""""""""""""""""""""""""""Logger"""""""""""""""""""""


class Logger(object):
    def __init__(self):
        TXLog = Path(INPUT_DIRTX + '/PredictedTempTreat.txt')
        TXLog.touch(exist_ok=True)  # will create file, if it exists will do nothing
        self.terminal = sys.stdout
        self.log = open(TXLog, 'w+')

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass


In [7]:
"""""""""""""""""""""IMPORT AND GATHER DATA"""""""""""""""""""""
@numba.jit(parallel=True, cache=True, nogil=True)
def Import():
    lstFilesTR = glob2.glob(INPUT_DIRTR + '/**/*.csv')
    lstFilesTX1 = glob2.glob(INPUT_DIRTX + '/UMD/*.csv')
    lstFilesTX2 = glob2.glob(INPUT_DIRTX + '/Nagoya/*.csv')
    # DSF = ['EnergyDelivered','SonicationNum','Duration','PowerDelivered','PeakAvgTemp','PeakMaxTemp', 'AvgThickness', 'AvgOANGS', 'BkgdEliminationOnOff', 'AvgSkullFocalDistance', 'skullArea', 'NoElOn','AvgSDR_withnorm','SDRstd_withnorm','StdThickness','StdOANGS','AvgOANGSTh20','AvgOANGSTh25','OANGS','Freq','ThermometryFrequencyDirection','ThermometryOrientation','ThermometryType']
    DSF = ['EnergyDelivered', 'SonicationNum', 'Duration', 'PowerDelivered', 'PeakAvgTemp', 'PeakMaxTemp',
           'AvgThickness', 'AvgOANGS', 'BkgdEliminationOnOff', 'AvgSkullFocalDistance', 'skullArea', 'NoElOn',
           'AvgSDR_withnorm', 'SDRstd_withnorm', 'StdThickness', 'StdOANGS', 'AvgOANGSTh20', 'AvgOANGSTh25', 'OANGS',
           'Freq', 'EstimatedTemperature', 'ThermometryFrequencyDirection', 'ThermometryOrientation', 'ThermometryType']
    UI = pd.DataFrame(index=None)
    DS = pd.DataFrame(index=None)
    UD = pd.DataFrame(index=None)
    ED = pd.DataFrame(index=None)
    DT = pd.DataFrame(index=None)
    list1 = []
    return lstFilesTX1, lstFilesTX2, lstFilesTR, DS, UI, UD, ED, DT, list1, DSF

In [8]:
@numba.jit(parallel=True, cache=True, fastmath=True, nogil=True)
def Gather(lstFilesTR, DS, list1, DSF):
    for f in lstFilesTR:
        df = pd.read_csv(f, skipinitialspace=True, skiprows=[0], usecols=DSF)
        df.replace([np.inf, -np.inf], np.nan)
        df = df.dropna()
        df = df[
            ['SonicationNum', 'AvgThickness', 'AvgOANGS', 'BkgdEliminationOnOff', 'AvgSkullFocalDistance', 'skullArea',
             'NoElOn', 'Duration', 'PowerDelivered', 'EnergyDelivered', 'PeakAvgTemp', 'PeakMaxTemp', 'AvgSDR_withnorm',
             'SDRstd_withnorm', 'StdThickness', 'StdOANGS', 'AvgOANGSTh20', 'AvgOANGSTh25', 'OANGS', 'Freq',
             'EstimatedTemperature', 'ThermometryFrequencyDirection', 'ThermometryOrientation', 'ThermometryType']]
        list1.append(df)
    DS = pd.concat(list1, ignore_index=True)
    DS = DS.dropna()
    return DS

In [9]:
@numba.jit(parallel=True, fastmath=True, nogil=True)
def get_train_test(DS, test_size=0.001):
    """Split Data into train and test sets."""
    # Split-out validation dataset
    le = {'le1': LabelEncoder(),
          'le2': LabelEncoder()}
    DS[['ThermometryFrequencyDirection']] = DS[['ThermometryFrequencyDirection']].apply(le['le1'].fit_transform)
    DS[['ThermometryOrientation']] = DS[['ThermometryOrientation']].apply(le['le2'].fit_transform)
    MX = DS[['SonicationNum', 'BkgdEliminationOnOff', 'Duration', 'PowerDelivered', 'EnergyDelivered', 'AvgThickness',
             'AvgOANGS', 'AvgSkullFocalDistance', 'skullArea', 'NoElOn', 'AvgSDR_withnorm', 'SDRstd_withnorm',
             'StdThickness', 'StdOANGS', 'AvgOANGSTh20', 'AvgOANGSTh25', 'OANGS', 'Freq', 'EstimatedTemperature',
             'ThermometryFrequencyDirection', 'ThermometryOrientation']].applymap(np.log1p)
    MY = DS[['PeakAvgTemp','PeakMaxTemp']].applymap(np.log1p)
    X_train, X_test, y_train, y_test = train_test_split(MX, MY, test_size=test_size,random_state=9)
    return le, X_train, X_test, y_train, y_test

In [10]:
lstFilesTX1, lstFilesTX2, lstFilesTR, DS, UI, UD, ED, DT, list1, DSF = Import()

In [11]:
DS = Gather(lstFilesTR, DS, list1, DSF)

In [12]:
@numba.jit(parallel=True, fastmath=True, nogil=True)
def train(X_train, X_test, y_train, y_test):
    print('Training of the treatments performed so far')
    model=MultiverseRegressor()
    model.fit(X_train,y_train)
    P = model.predict(X_test)
    print('MAE: %0.3f (AVG RMSE %0.3f) [%s]' % (
    mean_absolute_error(y_test, P), np.sqrt(mean_squared_error(y_test, P)), 'ensemble'))
    model.partial_fit(X_test,y_test)
    return model

In [13]:
@numba.jit(parallel=True, fastmath=True, nogil=True)
def SIM(f, UI, UD, DSF, le, model, ED):
    list2 = []
    list3 = []
    UI = pd.read_csv(f, skipinitialspace=True, header=1, usecols=DSF)
    UI.replace([np.inf, -np.inf], np.nan)
    UI = UI.dropna()
    UI = UI[
            ['SonicationNum', 'AvgThickness', 'AvgOANGS', 'BkgdEliminationOnOff', 'AvgSkullFocalDistance', 'skullArea',
             'NoElOn', 'Duration', 'PowerDelivered', 'EnergyDelivered', 'PeakAvgTemp', 'PeakMaxTemp', 'AvgSDR_withnorm',
             'SDRstd_withnorm', 'StdThickness', 'StdOANGS', 'AvgOANGSTh20', 'AvgOANGSTh25', 'OANGS', 'Freq',
             'EstimatedTemperature', 'ThermometryFrequencyDirection', 'ThermometryOrientation']]
    UI[['ThermometryFrequencyDirection']] = UI[['ThermometryFrequencyDirection']].apply(le['le1'].transform)
    UI[['ThermometryOrientation']] = UI[['ThermometryOrientation']].apply(le['le2'].transform)
    new = UI[
            ['SonicationNum', 'BkgdEliminationOnOff', 'Duration', 'PowerDelivered', 'EnergyDelivered', 'AvgThickness',
             'AvgOANGS', 'AvgSkullFocalDistance', 'skullArea', 'NoElOn', 'AvgSDR_withnorm', 'SDRstd_withnorm',
             'StdThickness', 'StdOANGS', 'AvgOANGSTh20', 'AvgOANGSTh25', 'OANGS', 'Freq', 'EstimatedTemperature',
             'ThermometryFrequencyDirection', 'ThermometryOrientation']].applymap(np.log1p)
    ADAbostpred = model.predict(new)
    l = new.shape[0]
    UI['ADABOOST P_av_Temperature'] = np.expm1(ADAbostpred[:, 0])
    UI['ADABOOST P_Max_Temperature'] = np.expm1(ADAbostpred[:, 1])
    UD = pd.read_csv(f, skipinitialspace=True, header=1, usecols=DSF)
    UD.replace([np.inf, -np.inf], np.nan)
    UD = UD.dropna()
    UD.replace([np.inf, -np.inf], np.nan)
    UD = UD.dropna()
    #		UI = UI[['SonicationNum', 'AvgThickness', 'AvgOANGS','BkgdEliminationOnOff', 'AvgSkullFocalDistance','skullArea', 'NoElOn','Duration','LNS','LNP','LNE','PeakAvgTemp','PeakMaxTemp','EnergyDelivered','PowerDelivered','AvgSDR_withnorm','SDRstd_withnorm','StdThickness','StdOANGS','AvgOANGSTh20','AvgOANGSTh25','OANGS','Freq','ThermometryFrequencyDirection','ThermometryOrientation','ThermometryType','EstimatedTemperature']]
    UD = UD[
            ['SonicationNum', 'AvgThickness', 'AvgOANGS', 'BkgdEliminationOnOff', 'AvgSkullFocalDistance', 'skullArea',
             'NoElOn', 'Duration', 'PowerDelivered', 'EnergyDelivered', 'PeakAvgTemp', 'PeakMaxTemp', 'AvgSDR_withnorm',
             'SDRstd_withnorm', 'StdThickness', 'StdOANGS', 'AvgOANGSTh20', 'AvgOANGSTh25', 'OANGS', 'Freq',
             'EstimatedTemperature', 'ThermometryFrequencyDirection', 'ThermometryOrientation']]
    UD[['ThermometryFrequencyDirection']] = UD[['ThermometryFrequencyDirection']].apply(le['le1'].transform)
    UD[['ThermometryOrientation']] = UD[['ThermometryOrientation']].apply(le['le2'].transform)
    l = UD.shape[0]
    u = 1
    errorpred1 = []
    for z in range(l):
        if z == 0:
            XP = UD[['SonicationNum', 'BkgdEliminationOnOff', 'Duration', 'PowerDelivered', 'EnergyDelivered',
                     'AvgThickness', 'AvgOANGS', 'AvgSkullFocalDistance', 'skullArea', 'NoElOn', 'AvgSDR_withnorm',
                     'SDRstd_withnorm', 'StdThickness', 'StdOANGS', 'AvgOANGSTh20', 'AvgOANGSTh25', 'OANGS', 'Freq',
                     'EstimatedTemperature', 'ThermometryFrequencyDirection', 'ThermometryOrientation']].applymap(
                np.log1p).iloc[[z]]  # Parameters from treatment
            A = model.predict(XP)
            yfin = pd.Series(np.amin(A).tolist())
            list3.append(np.expm1(yfin))
        elif z == 1:
            XT = UD[['SonicationNum', 'BkgdEliminationOnOff', 'Duration', 'PowerDelivered', 'EnergyDelivered',
                     'AvgThickness', 'AvgOANGS', 'AvgSkullFocalDistance', 'skullArea', 'NoElOn', 'AvgSDR_withnorm',
                     'SDRstd_withnorm', 'StdThickness', 'StdOANGS', 'AvgOANGSTh20', 'AvgOANGSTh25', 'OANGS', 'Freq',
                     'EstimatedTemperature', 'ThermometryFrequencyDirection', 'ThermometryOrientation']].applymap(
                np.log1p).iloc[[(z - 1)]]  # Parameters from treatment
            XP = UD[['SonicationNum', 'BkgdEliminationOnOff', 'Duration', 'PowerDelivered', 'EnergyDelivered',
                     'AvgThickness', 'AvgOANGS', 'AvgSkullFocalDistance', 'skullArea', 'NoElOn', 'AvgSDR_withnorm',
                     'SDRstd_withnorm', 'StdThickness', 'StdOANGS', 'AvgOANGSTh20', 'AvgOANGSTh25', 'OANGS', 'Freq',
                     'EstimatedTemperature', 'ThermometryFrequencyDirection', 'ThermometryOrientation']].applymap(
                np.log1p).iloc[[z]]
            YT = UD[['PeakAvgTemp', 'PeakMaxTemp']].applymap(np.log1p).iloc[[(z - 1)]]  # temperature from treatment
            model.fit(XT,YT)
            A = model.predict(XP)
            yfin = pd.Series(np.amin(A).tolist())
            list3.append(np.expm1(yfin))
            ED = pd.concat(list3, ignore_index=True)
        else:
            XP = UD[['SonicationNum', 'BkgdEliminationOnOff', 'Duration', 'PowerDelivered', 'EnergyDelivered',
                     'AvgThickness', 'AvgOANGS', 'AvgSkullFocalDistance', 'skullArea', 'NoElOn', 'AvgSDR_withnorm',
                     'SDRstd_withnorm', 'StdThickness', 'StdOANGS', 'AvgOANGSTh20', 'AvgOANGSTh25', 'OANGS', 'Freq',
                     'EstimatedTemperature', 'ThermometryFrequencyDirection', 'ThermometryOrientation']].applymap(
                np.log1p).iloc[[z]]
            XT1 = UD[['SonicationNum', 'BkgdEliminationOnOff', 'Duration', 'PowerDelivered', 'EnergyDelivered',
                      'AvgThickness', 'AvgOANGS', 'AvgSkullFocalDistance', 'skullArea', 'NoElOn', 'AvgSDR_withnorm',
                      'SDRstd_withnorm', 'StdThickness', 'StdOANGS', 'AvgOANGSTh20', 'AvgOANGSTh25', 'OANGS',
                      'Freq', 'EstimatedTemperature', 'ThermometryFrequencyDirection',
                      'ThermometryOrientation']].applymap(np.log1p).iloc[[(z - 1)]]
            YT1 = UD[['PeakAvgTemp', 'PeakMaxTemp']].applymap(np.log1p).iloc[
                [(z - 1)]]  # temperature from treatment
            model.fit(XT1,YT1)
            A = model.predict(XP)
            yfin = pd.Series(np.amin(A).tolist())
            list3.append(np.expm1(yfin))
            ED = pd.concat(list3, ignore_index=True)
    # --------------------------------Print Out the Data---------------------------------------------#
    print(f)
    print(
        '      Parameters     , Predictions Prior to, Beginning treatment,    FUS WS CURRENT PRED  ,     adaptive TX Prediction ,        Actual Temp by cursor, ')
    print('SON#,Energy,PredT lowlim,PredT Uplim,FusWsPred,DuringTXPred,Actual Temp,')
    for s, d, l, e, ws, ed, lp in zip(UI['SonicationNum'], UI['EnergyDelivered'], UI['ADABOOST P_av_Temperature'],
                                      UI['ADABOOST P_Max_Temperature'], UI['EstimatedTemperature'], ED,
                                      UI['PeakAvgTemp']):
        print(int(s), ',', int(d), ',', (l), ',', (e), ',', (ws), ',', ed, ',', (lp), ',')
    print('Final MAE score using Original: [%.8f]' % mae(UI['PeakAvgTemp'].values,
                                                                         UI['EstimatedTemperature'].values))
    print('Final MAE score using  PreTX Stack: [%.8f]' % mae(UI['PeakAvgTemp'].values,
                                                                             UI['ADABOOST P_av_Temperature'].values))
    print('Final MAE score using AdaptiveTX Stack: [%.8f]' % mae(UI['PeakAvgTemp'].values, ED.values))
    print('Final MSE score using Original: [%.8f]' % mse(UI['PeakAvgTemp'].values,
                                                                        UI['EstimatedTemperature'].values))
    print('Final MSE score using  PreTX Stack: [%.8f]' % mse(UI['PeakAvgTemp'].values,
                                                                            UI['ADABOOST P_av_Temperature'].values))
    print('Final MSE score using AdaptiveTX Stack: [%.8f]' % mse(UI['PeakAvgTemp'].values, ED.values))
    print('Final RMSE score using Original: [%.8f]' % rmse(UI['PeakAvgTemp'].values, UI['EstimatedTemperature'].values))
    print('Final RMSE score using  PreTX Stack: [%.8f]' % rmse(UI['PeakAvgTemp'].values, UI['ADABOOST P_av_Temperature'].values))
    print('Final RMSE score using AdaptiveTX Stack: [%.8f]' % rmse(UI['PeakAvgTemp'].values, ED.values))
    ### Using Bootstraped Extremely randomized trees to create statistical distribution and output mean and standard deviation, values not exactly the first 50c or
    ### or 54c since this could be treatment technique so it is the average location of multiple 50c to make more stringent
    IV = ETR(n_estimators=30, bootstrap=True,n_jobs=-1).fit(UI[['ADABOOST P_av_Temperature']].apply(np.log1p).values,
                                       UI[['EnergyDelivered']].apply(np.log1p).values)
    UV = ETR(n_estimators=30,bootstrap=True, n_jobs=-1).fit(UI[['PeakAvgTemp']].apply(np.log1p).values,
                                       UI[['EnergyDelivered']].apply(np.log1p).values)
    score = IV.predict([[np.log1p(50)]], return_std=True)
    score2 = UV.predict([[np.log1p(50)]], return_std=True)
    print('Energy to reach 50c: %0.3f +/- %0.3f [%s] ' % (
    np.expm1(score[0]), np.expm1(score[1]), 'std of pretreatment Temp predictions'))
    print('Energy to reach 50c: %0.3f +/- %0.3f [%s] ' % (
    np.expm1(score2[0]), np.expm1(score2[1]), 'std of actual Temp measurements'))
    score = IV.predict([[np.log1p(54)]], return_std=True)
    score2 = UV.predict([[np.log1p(54)]], return_std=True)
    print('Energy to reach 54c: %0.3f +/- %0.3f [%s] ' % (
    np.expm1(score[0]), np.expm1(score[1]), 'std of pretreatment Temp predictions'))
    print('Energy to reach 54c: %0.3f +/- %0.3f [%s] ' % (
    np.expm1(score2[0]), np.expm1(score2[1]), 'std of actual Temp measurements'))

In [14]:

le, X_train, X_test, y_train, y_test = get_train_test(DS, 0.5)

In [15]:
%time stacks = train(X_train, X_test, y_train, y_test)

Training of the treatments performed so far
MAE: 0.042 (AVG RMSE 0.102) [ensemble]
CPU times: user 21min 58s, sys: 1min 30s, total: 23min 29s
Wall time: 9min 15s


In [16]:
Parallel(n_jobs=1, prefer="threads")(delayed(SIM)(f, UI, UD, DSF, le, stacks, ED) for f in lstFilesTX2)
Parallel(n_jobs=1, prefer="threads")(delayed(SIM)(f, UI, UD, DSF, le, stacks, ED) for f in lstFilesTX1)
#runInParallel(SIM(lstFilesTX2, UI, UD, DSF, le, stacks, ED),SIM(lstFilesTX1, UI, UD, DSF, le, stacks, ED))

/home/osboxes/GBMon/DB/Test2/Nagoya/ET007-425012-171125.csv
      Parameters     , Predictions Prior to, Beginning treatment,    FUS WS CURRENT PRED  ,     adaptive TX Prediction ,        Actual Temp by cursor, 
SON#,Energy,PredT lowlim,PredT Uplim,FusWsPred,DuringTXPred,Actual Temp,
1 , 1988 , 43.73820297545652 , 44.92833197390621 , 42 , 43.73820297545652 , 43.553528 ,
2 , 2823 , 45.08512178174472 , 49.34331179403698 , 49 , 45.08512178174472 , 44.687393 ,
3 , 4027 , 47.896532914538405 , 49.828786779113614 , 51 , 47.89638252860915 , 47.970797999999995 ,
4 , 4028 , 46.50440365890136 , 49.781800742705165 , 51 , 46.50843019851509 , 47.635933 ,
5 , 7709 , 52.4457026081146 , 57.4379165588178 , 61 , 52.44570388235948 , 55.005455000000005 ,
6 , 9918 , 53.61832523362428 , 59.680461932125745 , 65 , 53.63510017540497 , 52.126205 ,
7 , 14867 , 56.12660075311087 , 58.25851116813152 , 67 , 56.08911435542177 , 56.959324 ,
8 , 17094 , 56.69490219441458 , 58.798713882730596 , 65 , 56.688319170188706 ,

Energy to reach 50c: 10280.699 +/- 0.936 [std of pretreatment Temp predictions] 
Energy to reach 50c: 14829.227 +/- 0.793 [std of actual Temp measurements] 
Energy to reach 54c: 16710.567 +/- 0.361 [std of pretreatment Temp predictions] 
Energy to reach 54c: 21909.214 +/- 0.283 [std of actual Temp measurements] 
/home/osboxes/GBMon/DB/Test2/Nagoya/ET007-425-180425-0022.csv
      Parameters     , Predictions Prior to, Beginning treatment,    FUS WS CURRENT PRED  ,     adaptive TX Prediction ,        Actual Temp by cursor, 
SON#,Energy,PredT lowlim,PredT Uplim,FusWsPred,DuringTXPred,Actual Temp,
1 , 1634 , 43.66178582453368 , 45.24092209704807 , 41 , 43.66178582453368 , 44.374992 ,
2 , 2047 , 45.26585234291406 , 49.39083882017984 , 48 , 45.26585234291406 , 47.000561 ,
3 , 2048 , 45.99673157068902 , 48.93379521701231 , 49 , 46.44697628032042 , 45.530396 ,
4 , 2469 , 46.798843189155576 , 50.375809429789896 , 50 , 46.793357405898576 , 47.765029999999996 ,
5 , 3266 , 49.63200509261188 , 51.8

Energy to reach 50c: 9197.881 +/- 0.565 [std of pretreatment Temp predictions] 
Energy to reach 50c: 18099.506 +/- 0.557 [std of actual Temp measurements] 
Energy to reach 54c: 23497.579 +/- 0.112 [std of pretreatment Temp predictions] 
Energy to reach 54c: 20512.423 +/- 0.389 [std of actual Temp measurements] 
/home/osboxes/GBMon/DB/Test2/Nagoya/ET007-425-180207-0011.csv
      Parameters     , Predictions Prior to, Beginning treatment,    FUS WS CURRENT PRED  ,     adaptive TX Prediction ,        Actual Temp by cursor, 
SON#,Energy,PredT lowlim,PredT Uplim,FusWsPred,DuringTXPred,Actual Temp,
1 , 2432 , 40.40595540152371 , 41.01573096900927 , 42 , 40.40595540152371 , 40.210403 ,
2 , 5490 , 44.063428397109085 , 46.84989889849013 , 46 , 44.06338757008501 , 44.627102 ,
3 , 6646 , 45.441312149791074 , 47.31943070093428 , 48 , 45.43915528239991 , 45.484161 ,
4 , 9930 , 47.22459932452948 , 50.04311606632344 , 52 , 47.215106657215415 , 47.293659000000005 ,
5 , 13476 , 48.57859006896216 , 50.1

Energy to reach 54c: 14457.282 +/- 0.422 [std of pretreatment Temp predictions] 
Energy to reach 54c: 17847.604 +/- 0.464 [std of actual Temp measurements] 
/home/osboxes/GBMon/DB/Test2/Nagoya/ET007-425011-171028.csv
      Parameters     , Predictions Prior to, Beginning treatment,    FUS WS CURRENT PRED  ,     adaptive TX Prediction ,        Actual Temp by cursor, 
SON#,Energy,PredT lowlim,PredT Uplim,FusWsPred,DuringTXPred,Actual Temp,
1 , 3213 , 43.87318707153136 , 44.659097451444694 , 45 , 43.87318707153136 , 41.824715000000005 ,
2 , 4375 , 42.88569389075997 , 45.29608285611104 , 45 , 42.885599722377975 , 42.708519 ,
3 , 5937 , 44.152701233080805 , 45.93847200950755 , 46 , 44.15271307484951 , 43.601509 ,
4 , 8199 , 46.841444950619135 , 48.22420993601479 , 48 , 46.8441574431141 , 45.697128 ,
5 , 10696 , 46.88269417774998 , 48.35618849087412 , 49 , 46.733016633813406 , 48.03764 ,
6 , 17302 , 50.49576109804369 , 55.56924724973487 , 57 , 50.49576109804369 , 50.88063 ,
7 , 25524 , 51.03

/home/osboxes/GBMon/DB/Test2/Nagoya/ET007-425-180919.csv
      Parameters     , Predictions Prior to, Beginning treatment,    FUS WS CURRENT PRED  ,     adaptive TX Prediction ,        Actual Temp by cursor, 
SON#,Energy,PredT lowlim,PredT Uplim,FusWsPred,DuringTXPred,Actual Temp,
1 , 1500 , 41.04199159131358 , 42.048545068761804 , 39 , 41.04199159131358 , 38.224777 ,
2 , 2208 , 41.98640092588257 , 44.29697341212257 , 39 , 41.986344557758066 , 39.848728 ,
3 , 3519 , 43.07906780505418 , 44.521964134231865 , 41 , 43.10120798528038 , 40.411568 ,
4 , 6044 , 44.03044362926626 , 46.22226527582607 , 43 , 43.967977160329085 , 43.116879 ,
5 , 9000 , 46.012923190067966 , 49.115563224270964 , 46 , 45.97991155353241 , 44.617619 ,
6 , 13504 , 47.33031164998931 , 49.97907953059165 , 49 , 47.14593116706324 , 46.489498 ,
7 , 20025 , 50.222918186680474 , 50.65998165612339 , 52 , 50.22664679721703 , 49.107966999999995 ,
8 , 30094 , 51.28400799564417 , 54.53293858158174 , 56 , 51.301609720948086 , 50.014

...

Energy to reach 50c: 16665.777 +/- 0.688 [std of pretreatment Temp predictions] 
Energy to reach 50c: 29760.102 +/- 0.200 [std of actual Temp measurements] 
Energy to reach 54c: 21491.440 +/- 0.356 [std of pretreatment Temp predictions] 
Energy to reach 54c: 33422.771 +/- 0.162 [std of actual Temp measurements] 
/home/osboxes/GBMon/DB/Test2/UMD/BR007-112-180406.csv
      Parameters     , Predictions Prior to, Beginning treatment,    FUS WS CURRENT PRED  ,     adaptive TX Prediction ,        Actual Temp by cursor, 
SON#,Energy,PredT lowlim,PredT Uplim,FusWsPred,DuringTXPred,Actual Temp,
1 , 870 , 40.98404508277477 , 42.420366162408726 , 38 , 40.98404508277477 , 39.596752 ,
2 , 1718 , 42.984478826439414 , 46.29016439083965 , 43 , 42.97586168667051 , 44.830498 ,
3 , 1717 , 45.034983222178575 , 46.427453108216184 , 47 , 45.109637127362944 , 43.941269 ,
4 , 1717 , 44.66232090305472 , 47.02837044690883 , 46 , 44.44912098217 , 39.160446 ,
5 , 2163 , 45.822138205084144 , 46.82698852288429 , 46

Energy to reach 50c: 3237.107 +/- 0.164 [std of pretreatment Temp predictions] 
Energy to reach 50c: 2805.299 +/- 0.249 [std of actual Temp measurements] 
Energy to reach 54c: 5563.267 +/- 0.105 [std of pretreatment Temp predictions] 
Energy to reach 54c: 5022.240 +/- 0.104 [std of actual Temp measurements] 
/home/osboxes/GBMon/DB/Test2/UMD/BR007-112-171129B.csv
      Parameters     , Predictions Prior to, Beginning treatment,    FUS WS CURRENT PRED  ,     adaptive TX Prediction ,        Actual Temp by cursor, 
SON#,Energy,PredT lowlim,PredT Uplim,FusWsPred,DuringTXPred,Actual Temp,
1 , 2374 , 43.56181120098415 , 44.710175790333814 , 42 , 43.56181120098415 , 42.896133 ,
2 , 2375 , 42.00186293052066 , 45.093280759218445 , 44 , 42.00186293052066 , 42.682503000000004 ,
3 , 2376 , 44.3137343137614 , 45.11174350979576 , 43 , 44.313634920417826 , 45.392582 ,
4 , 2375 , 44.028112883495965 , 45.5564899685119 , 45 , 44.591257503049604 , 43.510185 ,
5 , 2372 , 43.96681821341211 , 46.012879475946

Energy to reach 54c: 10747.448 +/- 0.252 [std of pretreatment Temp predictions] 
Energy to reach 54c: 10130.343 +/- 0.130 [std of actual Temp measurements] 
/home/osboxes/GBMon/DB/Test2/UMD/BR007-112-180718.csv
      Parameters     , Predictions Prior to, Beginning treatment,    FUS WS CURRENT PRED  ,     adaptive TX Prediction ,        Actual Temp by cursor, 
SON#,Energy,PredT lowlim,PredT Uplim,FusWsPred,DuringTXPred,Actual Temp,
1 , 1716 , 41.093445080362095 , 43.17449527429971 , 40 , 41.093445080362095 , 43.363106 ,
2 , 2150 , 44.6185336465119 , 46.099025767890566 , 49 , 44.6185336465119 , 45.558417999999996 ,
3 , 2152 , 44.214052009911015 , 46.57991243782621 , 48 , 44.18597593516383 , 44.511848 ,
4 , 3019 , 46.50288261052898 , 50.19748587451681 , 50 , 46.50288261052898 , 48.18803 ,
5 , 4287 , 49.73482185171641 , 52.31525070204126 , 53 , 49.73482185171641 , 51.208359 ,
6 , 4286 , 50.55603489725036 , 54.15298289936365 , 54 , 50.55451195127963 , 50.851147 ,
7 , 5176 , 50.274439287812

Energy to reach 50c: 1574.552 +/- 0.529 [std of pretreatment Temp predictions] 
Energy to reach 50c: 1723.164 +/- 0.464 [std of actual Temp measurements] 
Energy to reach 54c: 1574.552 +/- 0.529 [std of pretreatment Temp predictions] 
Energy to reach 54c: 1723.164 +/- 0.464 [std of actual Temp measurements] 
/home/osboxes/GBMon/DB/Test2/UMD/BR007-112-170828.csv
      Parameters     , Predictions Prior to, Beginning treatment,    FUS WS CURRENT PRED  ,     adaptive TX Prediction ,        Actual Temp by cursor, 
SON#,Energy,PredT lowlim,PredT Uplim,FusWsPred,DuringTXPred,Actual Temp,
1 , 1910 , 42.33310811946904 , 43.274919373022485 , 41 , 42.33310811946904 , 42.293613 ,
2 , 2406 , 43.52955031718566 , 45.74470466804155 , 44 , 43.52955137885288 , 42.827740000000006 ,
3 , 2882 , 44.23152739954452 , 47.18467765392911 , 45 , 44.23488998710709 , 43.830811 ,
4 , 3313 , 45.948343512739726 , 48.93599057391741 , 46 , 45.94830097800107 , 44.79586 ,
5 , 3811 , 46.01249389673655 , 49.11599456630487 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]